In [10]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

from ipywidgets import IntProgress
from IPython.display import display

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\14675\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from pymystem3 import Mystem
def lemmatize(text):
    m = Mystem()
    lem_arr = m.lemmatize(text)
    return "".join(lem_arr)

In [12]:
#Считываем золотые записи
df_m = pd.read_csv(r"vr_golden_records.csv", 
                 on_bad_lines='warn', 
                 delimiter=';',
                 na_values='',
                 dtype='str'
                )
print(df_m.columns)
df_m = df_m.drop(columns=['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])
df_m = df_m.rename(columns={'Код':'id', 'Наименование':'name', 'НТД':'ntd', 'НТД на материал':'ntd_m', 'Классификатор ЕНС':'ens', 'Классификатор ЕНС: Код':'ens_id'})
#Добавляем колонку, сигнализирующую, что это master-данные
df_m['source'] = 'master'
df_m

Index(['Код', 'Наименование', 'НТД', 'НТД на материал', 'Классификатор ЕНС',
       'Классификатор ЕНС: Код', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')


,id,name,ntd,ntd_m,ens,ens_id,source
0,1000134845,"Лента 65Г-0,1х90 ГОСТ 2283-79",ГОСТ 2283-79,NaN,"Ленты, широкополосный прокат",01.01.01.04,master
1,1000200838,Кислота фтористоводородная техническая А ГОСТ ...,ГОСТ 2567-89,NaN,Кислоты и окислы,01.03.06.01,master
2,1000421741,"Лента 65Г-Т-С-Н-1,7х70 ГОСТ 2283-79",ГОСТ 2283-79,NaN,"Ленты, широкополосный прокат",01.01.01.04,master
3,1000405580,Лист БТ-БШ-ПН-О-1 ГОСТ 19904-90 / К270В-5-II-Г...,ГОСТ 19904-90,ГОСТ 16523-97,ХК-листовой прокат,01.01.01.02,master
4,1000191549,Лист ВТ-АШ-ВД-НО-ПУ-4х600х2000 ГОСТ 19904-90 /...,ГОСТ 19904-90,ГОСТ 7350-77,ХК-листовой прокат,01.01.01.02,master
...,...,...,...,...,...,...,...
795860,1000266412,Сапоги мужские для защиты от механических возд...,ГОСТ 12.4.162-85,NaN,Обувь кожаная,07.01.03.02,master
795861,1000798742,"Халат рабочий мужской Бязь арт.Хал 003, матери...",ТО 14.12.30-003-70107142-2021,NaN,Халаты,07.01.04.06,master
795862,1000266420,Сапоги женские для защиты от общих производств...,ГОСТ Р 12.4.187-97,NaN,Обувь кожаная,07.01.03.02,master
795863,1000266481,Ботинки мужские для защиты от общих производст...,ГОСТ Р 12.4.187-97,NaN,Обувь кожаная,07.01.03.02,master


In [13]:
#Считываем записи для сопоставления
df_c = pd.read_csv(r"vr_candidates.csv", 
                 on_bad_lines='warn', 
                 delimiter=';',
                 na_values='',
                 dtype='str'
                )
print(df_c.columns)
df_c = df_c.rename(columns={'артикул':'id', 'Краткое наименование':'name', 'ЕИ':'unit'})
#Единица измерения могла бы пригодиться, если бы в эталонных данных она содержалась, поэтому придётся удалить
df_c = df_c.drop(columns=['unit'])
#Добавляем колонку, сигнализирующую, что это master-данные и пустую колонку для ГОСТа
df_c['source'] = 'candidate'
df_c['ntd'] = ''
df_c

Index(['артикул', 'Краткое наименование', 'ЕИ'], dtype='object')


,id,name,source,ntd
0,9716210026.002,"КАРАНДАШ АЛМ. 3908-0052 ГОСТ 607-80,",candidate,
1,9716220023.003,"КАРАНДАШ АЛМ. 3908-0083 ГОСТ 607-80 1карат,",candidate,
2,0657502328.000,"ПЛАСТИНА Ф-4 ""ПН"" 200Х200Х20 ПЕРВЫЙ СОРТ ТУ6-0...",candidate,
3,0494502255.000,ПРУТКИ БРОН3ОВЫЕ ПКРНХ-90-НД БРАЖ9-4 Г1628-78/...,candidate,
4,0477602023.000,ПРУТКИ И3 КАДМИЕВОЙ МЕДИ ТЯНУТЫЕ КР20.0 МК ТУ4...,candidate,
...,...,...,...,...
28874,7914370148.000,"УПЛОТНИТЕЛЬ,",candidate,
28875,9990297149.000,"ФИЛЬТР ВОЗДУШНЫЙ,",candidate,
28876,9990297149.200,"Фильтр салонный,",candidate,
28877,9990297210.000,"ФИЛЬТР ТОПЛИВНЫЙ,",candidate,


In [14]:
#Справочно формируем список всех ГОСТов, на всякий случай удаляем пробелы
df_m['ntd'] = df_m['ntd'].astype(str)
gost_list = df_m['ntd'].unique()
print (gost_list.size)
gost_list_no_spaces = df_m['ntd'].str.replace(' ', '', regex=False).unique()
print(gost_list_no_spaces.size)

8233
8233


In [35]:
#Объединяем для последующей конкатенации, формируем мешок слов и векторизируем
df = pd.concat([df_m, df_c], ignore_index=True)
print (len(df))

824744


In [49]:
#Основная лемматизация с прогресс баром и промежуточной записью в файл по 1000 элементов
count = 1000
start_loc = 536000
max_count = len(df)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar
while start_loc <= max_count:
    print(str(start_loc)+' : '+str(start_loc+count))
    new_df = df.iloc[start_loc:start_loc+count].copy()  
    new_df['lemmatized_name'] = new_df['name'].apply(lemmatize)
    new_df.to_csv('lemmatized'+str(start_loc)+'.csv', encoding='utf-8-sig', sep=';')
    start_loc += count
    f.value += count # signal to increment the progress bar

IntProgress(value=0, max=824744)

0 : 1000
1000 : 2000
2000 : 3000
3000 : 4000
4000 : 5000
5000 : 6000
6000 : 7000
7000 : 8000
8000 : 9000
9000 : 10000
10000 : 11000
11000 : 12000
12000 : 13000
13000 : 14000
14000 : 15000
15000 : 16000
16000 : 17000
17000 : 18000
18000 : 19000
19000 : 20000
20000 : 21000
21000 : 22000
22000 : 23000
23000 : 24000
24000 : 25000
25000 : 26000
26000 : 27000
27000 : 28000
28000 : 29000
29000 : 30000
30000 : 31000
31000 : 32000
32000 : 33000
33000 : 34000
34000 : 35000
35000 : 36000
36000 : 37000
37000 : 38000
38000 : 39000
39000 : 40000
40000 : 41000
41000 : 42000
42000 : 43000
43000 : 44000
44000 : 45000
45000 : 46000
46000 : 47000
47000 : 48000
48000 : 49000
49000 : 50000
50000 : 51000
51000 : 52000
52000 : 53000
53000 : 54000
54000 : 55000
55000 : 56000
56000 : 57000
57000 : 58000
58000 : 59000
59000 : 60000
60000 : 61000
61000 : 62000
62000 : 63000
63000 : 64000
64000 : 65000
65000 : 66000
66000 : 67000
67000 : 68000
68000 : 69000
69000 : 70000
70000 : 71000
71000 : 72000
72000 : 73000


"""
#Пробная лемматизация на 50 записей для отработки процесса
print(df.loc[1,'name'])
print(lemmatize(df.loc[1,'name']))

max_count = 50

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

new_df = df.iloc[824740:824748].copy()  
new_df['lemmatized_name'] = new_df['name'].apply(lemmatize)
new_df.to_csv('lemmatized_2.csv', encoding='utf-8-sig', sep=';')
f.value += 20
new_df
"""